У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_curve, auc, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns


In [ ]:
cust_seg = pd.read_csv('/content/customer_segmentation_train.csv')

In [ ]:
cust_seg.dtypes

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object

In [ ]:
cust_seg

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [ ]:
cust_seg['Var_1'].unique()

array(['Cat_4', 'Cat_6', 'Cat_7', 'Cat_3', 'Cat_1', 'Cat_2', nan, 'Cat_5'],
      dtype=object)

In [ ]:
cust_seg.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

Потрібно заповнити значення у колонках Work_Experience, Family_Size, Ever_Married, Profession, Graduated та Var_1

In [ ]:
cust_seg['Work_Experience'].value_counts()

Work_Experience
1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: count, dtype: int64

Колонка Work_Experience має близько 10% пропущених значень


In [ ]:
cust_seg['Work_Experience'].value_counts()

Work_Experience
1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: count, dtype: int64

In [ ]:
cust_seg['Var_1'].value_counts()

Var_1
Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: count, dtype: int64

In [ ]:
cust_seg['Family_Size'].value_counts()

Family_Size
2.0    2390
3.0    1497
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64

In [ ]:
cust_seg['Ever_Married'].value_counts()

Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64

In [ ]:
cust_seg['Graduated'].value_counts()

Graduated
Yes    4968
No     3022
Name: count, dtype: int64

In [ ]:
df_cust_seg = cust_seg.copy()

In [ ]:
cust_seg.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


Заповнюю у колонці Work_Experience 829 пропущених значень значенням середнім між медіаною та середнім значенням цілим числом 2.0

In [ ]:
df_cust_seg['Work_Experience'].fillna(value = 2.0, inplace = True)
df_cust_seg['Family_Size'].fillna(value = 3.0, inplace = True)
df_cust_seg['Ever_Married'].fillna(value = 'Yes', inplace = True)
df_cust_seg['Profession'].fillna(value = 'Unknown', inplace = True)
df_cust_seg['Graduated'].fillna(value = 'Yes', inplace = True)
df_cust_seg['Var_1'].fillna(value = 'Unknown', inplace = True)

Заповнюю у колонці Family_Size 335 пропущених значень числом 3, яке є медіаною та середнє число становить 2.85, що в цілому є нормальним значенням для цієї колонки

Пропущені значення у колонці Ever_Married думаю можна заповнити переважаючим значенням 'Yes', пропущені значення становлять 1,7% від загальної кількості, не сильно поміняється картина


Пропущені значення у колонці Profession заповню значенням 'Unknown'

Пропущені значення у колонці Graduated теж заповню переважаючим значенням 'Yes.

Пропущені значення у колонці Var_1 заповню значенням 'Unknown'

In [ ]:
df_cust_seg.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [ ]:
df_cust_seg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   object 
 5   Profession       8068 non-null   object 
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      8068 non-null   float64
 9   Var_1            8068 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [ ]:
df_cust_seg[:3]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,2.0,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B


In [ ]:
target = df_cust_seg['Segmentation']
train_df, test_df = train_test_split(df_cust_seg, test_size=0.2, random_state=42, stratify=target)

In [ ]:
# Створюю тренувальний і тестовий набори
input_cols = list(df_cust_seg.columns)[1:-1]
target_col = 'Segmentation'
train_inputs, train_targets = train_df[input_cols].copy(), train_df[target_col].copy()
test_inputs, test_targets = test_df[input_cols].copy(), test_df[target_col].copy()

#Визначаю числові та категоріальні колонки
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [ ]:
#encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#encoder.fit(train_inputs[categorical_cols])
#encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
#train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
#test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

Створюю інпути для OvR на оригінальних даних

In [ ]:
train_inputs_basic = train_inputs.copy()
test_inputs_basic = test_inputs.copy()

In [ ]:
scaler.fit(train_inputs_basic[numeric_cols])
train_inputs_basic[numeric_cols] = scaler.transform(train_inputs_basic[numeric_cols])
test_inputs_basic[numeric_cols] = scaler.transform(test_inputs_basic[numeric_cols])

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_inputs_basic[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
train_inputs_basic[encoded_cols] = encoder.transform(train_inputs_basic[categorical_cols])
test_inputs_basic[encoded_cols] = encoder.transform(test_inputs_basic[categorical_cols])

In [ ]:
train_inputs_basic = train_inputs_basic[numeric_cols + encoded_cols]
test_inputs_basic = test_inputs_basic[numeric_cols + encoded_cols]

Work_Experience**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTETomek
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [ ]:
from imblearn.over_sampling import SMOTENC


In [ ]:
#train test inputs for SMOTENC
train_inputs_smotenc = train_inputs.copy()
test_inputs_smotenc = test_inputs.copy()

In [ ]:
# Визначення категоріальних колонок
categorical_columns = train_inputs_smotenc.select_dtypes(include=['object']).columns
# Отримання індексів категоріальних колонок
categorical_indices = [train_inputs_smotenc.columns.get_loc(col) for col in categorical_columns]

numeric_columns = train_inputs_smotenc.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ])

In [ ]:
train_inputs_smotenc_encoded = preprocessor.fit_transform(train_inputs_smotenc)

In [ ]:
test_inputs_smotenc = preprocessor.fit_transform(test_inputs_smotenc)

In [ ]:
#train inputs оброблені SMOTENC
smotenc = SMOTENC(categorical_features=categorical_indices, random_state=0)
X_resampled, y_resampled = smotenc.fit_resample(train_inputs_smotenc_encoded, train_targets)

In [ ]:
X_resampled

<7256x24 sparse matrix of type '<class 'numpy.float64'>'
	with 48641 stored elements in Compressed Sparse Row format>

In [ ]:
#train test inputs for SMOTETOMEK
train_inputs_smotetomek = train_inputs.copy()
test_inputs_smotetomek = test_inputs.copy()

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_inputs_smotetomek[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
train_inputs_smotetomek[encoded_cols] = encoder.transform(train_inputs_smotetomek[categorical_cols])
test_inputs_smotetomek[encoded_cols] = encoder.transform(test_inputs_smotetomek[categorical_cols])

In [ ]:
train_inputs_smotetomek.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Gender_Female,...,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_Unknown
917,Female,No,0.197183,Yes,Artist,0.642857,Low,0.000,Cat_6,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3398,Male,Yes,0.760563,Yes,Entertainment,0.142857,Average,0.125,Cat_6,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2045,Female,No,0.211268,Yes,Entertainment,0.071429,Low,0.375,Cat_6,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8060,Female,Yes,0.422535,Yes,Artist,0.000000,Average,0.625,Cat_6,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4604,Female,Yes,0.140845,No,Doctor,0.642857,Low,0.000,Cat_7,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
train_inputs_smotetomek = train_inputs_smotetomek[numeric_cols + encoded_cols]

In [ ]:
smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(train_inputs_smotetomek, train_targets)

clf_smotetomek = SVC(kernel='linear',probability=True)
clf_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)

SVC(kernel='linear', probability=True)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [ ]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)


In [ ]:
ovr_model.fit(train_inputs_basic, train_targets)
ovr_predictions_basic = ovr_model.predict(test_inputs_basic)

print(classification_report(test_targets, ovr_predictions_basic))

              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.42      0.17      0.24       372
           C       0.49      0.63      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.48      1614
weighted avg       0.50      0.52      0.50      1614



In [ ]:
ovr_model.fit(X_resampled, y_resampled)
ovr_predictions_SMOTENC = ovr_model.predict(test_inputs_smotenc)

print(classification_report(test_targets, ovr_predictions_SMOTENC))

              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.39      0.25      0.30       372
           C       0.50      0.57      0.53       394
           D       0.68      0.71      0.70       454

    accuracy                           0.51      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.51      0.51      0.50      1614



In [ ]:
ovr_model.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions_SMOTETOMEK = ovr_model.predict(test_inputs_basic)

print(classification_report(test_targets, ovr_predictions_SMOTETOMEK))

              precision    recall  f1-score   support

           A       0.42      0.49      0.45       394
           B       0.38      0.23      0.29       372
           C       0.49      0.58      0.53       394
           D       0.68      0.70      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.49      1614
weighted avg       0.50      0.51      0.50      1614



Важко тут вибрати метрику по якій оцінити якість моделей, моделі працюють однаково практично. Оскільки мало класів то можна проаналізувати, що клас В має найгірший precision та recall. Все ж я б вибрав Weighted avg, оскільки вона "зважено рахує" не зважаючи на не великий дисбаланс класів.
Найкраще спрацювала модель SMOTENC, на мою думку.

Можливо моделі практично однкаово спрацювали через не великий дисбаланс класів, що видно по кількості елементів у кожному класі.
